In [20]:
import json

# Load train and test data
with open('train.json', 'r', encoding='utf-8') as f:
    train = json.load(f)

with open('nemwaf.json', 'r', encoding='utf-8') as f:
    test = json.load(f)

# Extract payload and injection data
X_train = [item.get("payload", []) for item in train]
X_test = [item.get("payload", []) for item in test]
y_train = [item.get("injection", []) for item in train]
y_test = [item.get("injection", []) for item in test]

# Create a list to hold the combined JSON elements
combined_json = []

# Iterate over the elements and create combined JSON
for x_train, y_train in zip(X_train, y_train):
    combined_element = {
        "payload": x_train,
        "injection": y_train
    }
    combined_json.append(combined_element)

for x_test, y_test in zip(X_test, y_test):
    combined_element = {
        "payload": x_test,
        "injection": y_test
    }
    combined_json.append(combined_element)


dat=[item.get("payload", []) for item in combined_json]
inj=[item.get("injection", []) for item in combined_json]

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(dat,inj, test_size=0.2, random_state=42)

In [22]:
from keras.preprocessing.text import Tokenizer

from sklearn.preprocessing import LabelEncoder

# Создаем объект LabelEncoder и преобразуем значения y_train и y_test
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
# Создаем объект Tokenizer и преобразуем инъекцию в числовые данные
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_matrix(X_train, mode='binary')
X_test = tokenizer.texts_to_matrix(X_test, mode='binary')


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Создание объекта модели
lr = LogisticRegression(C=10.0)

# Обучение модели на обучающих данных
lr.fit(X_train, y_train)

# Прогнозирование меток классов на тестовых данных
y_pred = lr.predict(X_test)

# Оценка качества модели
from sklearn.metrics import accuracy_score

# Вычисление метрик качества
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Вывод метрик качества
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.9600272026598157
Precision: 0.9594224924012158
Recall: 0.9601520912547529
F1 score: 0.9597871531736982


/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight
import numpy as np
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

print(class_weights)

# Must be in dict format for scikitlearn
class_weights = {
    0: class_weights[0],
    1: class_weights[1],
    
}

print(class_weights)
model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=class_weights,
    ccp_alpha=0.0,
    max_samples=None
)

hyperparameters = {
    'n_estimators': [50, 75, 100, 125, 150, 175, 200]
}

[0.98608472 1.01431364]
{0: 0.9860847211355762, 1: 1.014313635318464}


In [25]:
rf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=2,
    verbose=1,
    n_jobs=-1  
)

In [26]:
rf.fit(X=X_train, y=y_train)

Fitting 2 folds for each of 7 candidates, totalling 14 fits


GridSearchCV(cv=2,
             estimator=RandomForestClassifier(class_weight={0: 0.9860847211355762,
                                                            1: 1.014313635318464}),
             n_jobs=-1,
             param_grid={'n_estimators': [50, 75, 100, 125, 150, 175, 200]},
             verbose=1)

In [27]:
models = {}
models['random_forest'] = rf.best_estimator_

In [28]:
y_pred=models['random_forest'].predict(X_test)
# Вычисление метрик качества
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Вывод метрик качества
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.9658455493426024
Precision: 0.9560554148666766
Recall: 0.9761216730038023
F1 score: 0.9659843467790489


In [29]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.preprocessing.text import Tokenizer
from keras import layers
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,GRU,Embedding,Dropout,Flatten,Conv1D,MaxPooling1D,add,GlobalAveragePooling1D,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D

model = Sequential([
    Conv1D(128, 16, padding='same', activation='relu', input_shape=(1000,1)),
    MaxPooling1D(2),
    Conv1D(64, 16, padding='same', activation='relu'),
    MaxPooling1D(2),
    Conv1D(32, 8, padding='same', activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Обучаем модель
model.fit(X_train, y_train, epochs=3, batch_size=32)


Epoch 1/3
1655/1655 [==============================] - 130s 78ms/step - loss: 0.1465 - accuracy: 0.9443
Epoch 2/3
1655/1655 [==============================] - 135s 82ms/step - loss: 0.0854 - accuracy: 0.9676
Epoch 3/3
1655/1655 [==============================] - 131s 79ms/step - loss: 0.0728 - accuracy: 0.9723


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Оцениваем точность модели на тестовых данных
X_test = np.array(X_test)
y_test = np.array(y_test)


# Получаем предсказания модели
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Вычисляем метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Вывод метрик качества
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


414/414 [==============================] - 6s 13ms/step
Accuracy: 0.9681880006045036
Precision: 0.9615961596159616
Recall: 0.9749049429657795
F1 score: 0.9682048183671926


In [32]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Обучение модели на обучающих данных
clf = LinearSVC(random_state=42)
clf.fit(X_train, y_train)

# Получение предсказаний на тестовых данных
y_pred = clf.predict(X_test)

/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [33]:
# Вычисление метрик качества
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Вывод метрик качества
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.9594982620522896
Precision: 0.9585421412300683
Recall: 0.96
F1 score: 0.9592705167173251


In [34]:
model = Sequential([
    Conv1D(128, 16, padding='same', activation='relu', input_shape=(1000, 1)),
    MaxPooling1D(2),
    Conv1D(64, 16, padding='same', activation='relu'),
    MaxPooling1D(2),
    Conv1D(32, 8, padding='same', activation='relu'),
    MaxPooling1D(2),
    GRU(32),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [35]:
model.fit(X_train, y_train, epochs=3, batch_size=32)


Epoch 1/3
1655/1655 [==============================] - 155s 93ms/step - loss: 0.6658 - accuracy: 0.5392
Epoch 2/3
1655/1655 [==============================] - 156s 94ms/step - loss: 0.6351 - accuracy: 0.5716
Epoch 3/3
1655/1655 [==============================] - 153s 93ms/step - loss: 0.3188 - accuracy: 0.8655


In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Оцениваем точность модели на тестовых данных
X_test = np.array(X_test)
y_test = np.array(y_test)


# Получаем предсказания модели
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Вычисляем метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Вывод метрик качества
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


414/414 [==============================] - 7s 16ms/step
Accuracy: 0.9094755931691099
Precision: 0.9129166026724006
Recall: 0.9040304182509505
F1 score: 0.9084517805288095
